In [52]:
import xml.etree.ElementTree as ET
import numpy as np
import os
files = [i for i in os.listdir() if i.endswith("xml")]

In [53]:
files

['x20000030.xml', 'x10000020.xml', 'x10000029_x.xml']

In [56]:
a = ""
b = np.repeat(a,10)

In [57]:
b

array(['', '', '', '', '', '', '', '', '', ''], dtype='<U1')

In [41]:

file = "x10000020"
tree = ET.parse(file+'.xml')
root = tree.getroot()
Target = root.find(".//experimentType").text
Target_kind = root.find(".//kind").text
Target_mode = root.find(".//mode").text
bibliography = root.find(".//bibliographyLink")
commonProp  = root.find(".//commonProperties")
dataGroup = root.findall(".//dataGroup")
for i in root.findall("*"):
    if "ignitionType" in i.tag:
        MeasurnmentType = root.find(".//ignitionType").attrib["target"]+root.find(".//ignitionType").attrib["type"]
    else:
        MeasurnmentType

In [18]:
properties = {}
concentartion_Dict = {}
unsrt = {}
datagroupDict = {}
Dict = {}
points = {}

In [31]:
for child in commonProp:
    #print(child.[@attribName]
    if child.tag == "property":
        if child.attrib["name"] == "initial composition":
            for zero in child:       
                temp = {}
                for minusOne in zero:
                    if minusOne.tag == "speciesLink":
                        temp["name"] = minusOne.attrib["preferredKey"]
                        temp["CAS"] = minusOne.attrib["CAS"]
                        temp["InChI"] = minusOne.attrib["InChI"]
                        temp["SMILES"] = minusOne.attrib["SMILES"]
                        temp["chemName"] = minusOne.attrib["chemName"]
                    elif minusOne.tag == "amount":
                        value = minusOne.text
                        temp["value"] = value
                        temp["unit"] = minusOne.attrib["units"]
                concentartion_Dict[temp["name"]] = temp
            properties["initial composition"] = concentartion_Dict
        if child.attrib["name"] == "evaluated standard deviation" or child.attrib["name"] == "uncertainty":
            type_ = child.attrib["reference"]
            unsrt_Dict = {}
            unsrt_Dict["type"] = child.attrib["kind"]
            if "method" in child.attrib:
                unsrt_Dict["method"] = child.attrib["method"]
            else:
                unsrt_Dict["method"] = "instrumental error"
                unsrt_Dict["bound"] = child.attrib["bound"]
                unsrt_Dict["label"] = child.attrib["label"]
            unsrt_Dict["sourcetype"] = child.attrib["sourcetype"]
            unsrt_Dict["units"] = child.attrib["units"]
            for zero in child:
                #print(zero.text)
                unsrt_Dict["value"] = zero.text

            unsrt[type_] = unsrt_Dict
            properties["Unsrt"] = unsrt
        if child.attrib["name"] == "pressure":
            temp = {}
            temp["label"] = child.attrib["label"]
            temp["unit"] = child.attrib["units"]
            temp["sourcetype"] = child.attrib["sourcetype"]
            for zero in child:
                if zero.tag == "value":
                    temp["value"] = zero.text
            properties["pressure"] = temp
        if child.attrib["name"] == "temperature":
            temp = {}
            temp["label"] = child.attrib["label"]
            temp["unit"] = child.attrib["units"]
            temp["sourcetype"] = child.attrib["sourcetype"]
            for zero in child:
                if zero.tag == "value":
                    temp["value"] = zero.text
            properties["temperature"] = temp
#print(unsrt)

{'ignition delay': {'type': 'relative', 'method': 'generic uncertainty', 'sourcetype': 'estimated', 'units': 'unitless', 'value': '0.1'}}


In [30]:
for group in dataGroup:
    tempID = {}
    temp = {}
    tempID["id"] = group.attrib["id"]
    for ind,child in enumerate(group):
        if child.tag == "property":
            if child.attrib["name"] == "composition":
                tem = {}
                for zero in child:
                    tem["specification"] = zero.attrib
                tem["name"] = child.attrib["name"]
                tem["id"] = child.attrib["id"]
                tem["label"] = child.attrib["label"]
                tem["sourcetype"] = child.attrib["sourcetype"]
                tem["units"] = child.attrib["units"]
                temp[child.attrib["id"]] = tem
            else:
                temp[child.attrib["id"]] = child.attrib
        if child.tag == "dataPoint":
            data = {}
            for zero in child:
                #print(zero.tag)
                data[zero.tag]=zero.text
            points[str(ind-2)] = data
    Dict["Variables"] = temp
    Dict["DataPoints"] = points
    datagroupDict[tempID["id"]] = Dict


In [21]:
# Extracting pressure
if "pressure" in properties:
    pressure = float(properties["pressure"]["value"])
    p_unit = properties["pressure"]["unit"]
    p_sigma = 0 #by default
    if "pressure" in unsrt:
        p_sigma = unsrt["pressure"]["value"]
else:
    pressure = []
    for var in datagroupDict["dg1"]["Variables"]:
        if datagroupDict["dg1"]["Variables"][var]["name"] == "pressure":
            p_unit = datagroupDict["dg1"]["Variables"][var]["units"] 
            for point in datagroupDict["dg1"]["DataPoints"]:
                pressure.append(datagroupDict["dg1"]["DataPoints"][point][var])
    if "pressure" in unsrt:
        p_sigma = unsrt["pressure"]["value"]

In [22]:
#Extracting initial composition
if "initial composition" in properties:
    bathgas = []
    bathgas_conc = []
    bathgas_unit = []
    #print(properties)
    for component in properties["initial composition"]:
        if component == "H2":
            fuel = properties["initial composition"][component]["name"]
            fuel_conc = float(properties["initial composition"][component]["value"])
            fuel_unit = properties["initial composition"][component]["unit"]
            continue
        elif component == "O2":
            oxidizer = properties["initial composition"][component]["name"]
            oxidizer_conc = float(properties["initial composition"][component]["value"])
            oxidizer_unit = properties["initial composition"][component]["unit"]
            continue
        else:
            bathgas.append(properties["initial composition"][component]["name"])
            bathgas_conc.append(float(properties["initial composition"][component]["value"]))
            bathgas_unit.append(properties["initial composition"][component]["unit"])
else:
    fuel_conc = []
    oxidizer_conc = []
    bathgas = []
    bathgas_conc = []
    for var in datagroupDict["dg1"]["Variables"]:
        if datagroupDict["dg1"]["Variables"][var]["name"] == "composition":
            if datagroupDict["dg1"]["Variables"][var]["specification"]["preferredKey"] == "H2":
                fuel = datagroupDict["dg1"]["Variables"][var]["specification"]["preferredKey"]
                fuel_unit = datagroupDict["dg1"]["Variables"][var]["units"]
                for point in datagroupDict["dg1"]["DataPoints"]:
                    fuel_conc.append(float(datagroupDict["dg1"]["DataPoints"][point][var]))
            elif datagroupDict["dg1"]["Variables"][var]["specification"]["preferredKey"] == "O2":
                oxidizer = datagroupDict["dg1"]["Variables"][var]["specification"]["preferredKey"]
                oxidizer_unit = datagroupDict["dg1"]["Variables"][var]["units"]

                for point in datagroupDict["dg1"]["DataPoints"]:
                    oxidizer_conc.append(float(datagroupDict["dg1"]["DataPoints"][point][var]))
            else:
                bathgas.append(datagroupDict["dg1"]["Variables"][var]["specification"]["preferredKey"])
                bathgas_unit = datagroupDict["dg1"]["Variables"][var]["units"]

                temp_list = []
                for point in datagroupDict["dg1"]["DataPoints"]:
                    temp_list.append(float(datagroupDict["dg1"]["DataPoints"][point][var]))
                bathgas_conc.append(temp_list)

In [23]:
#Extracting temperature
if "temperature" in properties:
    temperature = float(properties["temperature"]["value"])
    t_unit = properties["temperature"]["unit"]
    t_sigma = 0 #by default
    if "temperature" in unsrt:
        t_sigma = unsrt["temperature"]["value"]
else:
    temperature = []
    #print(datagroupDict['dg1']["Variables"])
    for var in datagroupDict["dg1"]["Variables"]:
        if datagroupDict["dg1"]["Variables"][var]["name"] == "temperature":
            t_unit = datagroupDict["dg1"]["Variables"][var]["units"] 
            for point in datagroupDict["dg1"]["DataPoints"]:
                temperature.append(float(datagroupDict["dg1"]["DataPoints"][point][var]))
    if "temperature" in unsrt:
        t_sigma = unsrt["temperature"]["value"]

In [34]:
#Extracting experimental targets
exp_target = []
if Target_kind == "flame" and Target_mode == "premixed":
    exp = 'laminar burning velocity'
    Simulation_type = "UnstretchedPremixed"
elif Target_kind == "shock tube":
    exp = "ignition delay"
    Simulation_type = "isochor"
for var in datagroupDict['dg1']['Variables']:
    if exp in datagroupDict['dg1']['Variables'][var]['name']:
        exp_unit = datagroupDict['dg1']['Variables'][var]['units']
        for point in datagroupDict["dg1"]["DataPoints"]:
            exp_target.append(float(datagroupDict["dg1"]["DataPoints"][point][var]))
if exp in unsrt:
    exp_sigma = float(unsrt[exp]["value"])
    exp_sigma_type = unsrt[exp]["type"]
ndpt = len(datagroupDict["dg1"]["DataPoints"])

In [35]:
Input_units = "{}\t{}\t{}\t{}".format(fuel_unit,p_unit,t_unit,exp_unit)
Input_data_structure = ""
for i in datagroupDict['dg1']["Variables"]:
    Input_data_structure +=datagroupDict['dg1']["Variables"][i]["label"].strip("[]")+"\t"

In [36]:
#Input_data_structure

'T\ttau\t'

In [37]:
pressure = np.repeat(pressure,ndpt)

In [39]:
if exp_sigma_type == "relative":
    exp_std_dnvt = exp_sigma*np.asarray(exp_target)
elif exp_sigma_type == "absolute":
    exp_std_dnvt = np.repeat(exp_sigma,ndpt)